In [34]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [35]:
!pip install tensorflow_io
!pip install pydicom

In [36]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_io as tfio

from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.densenet import DenseNet121
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras import backend as K

from keras.models import load_model
from pydicom import dcmread
import pydicom

# TFRECORDS CREATION

In [37]:
base_path = '/content/drive/My Drive/'
df = pd.read_excel(base_path+"/X-RAY/tmp_df.xls")


In [38]:
test = "X-Ray_999"
df[df["ID protocol"] == test].iloc[:, -12:].values

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

In [39]:
import os
from fnmatch import fnmatch


pattern = "*.dcm"
file_list=[]
for path, subdirs, files in os.walk(base_path+"X-RAY/"):
  print(path, subdirs, "files",files)
  for idx,name in enumerate(files):
      if (fnmatch(name, pattern)):
          file_list.append(os.path.join(path, name))


/content/drive/My Drive/X-RAY/ ['X-Ray_1001', 'X-Ray_998', 'X-Ray_1002', 'X-Ray_1000', 'X-Ray_999'] files ['tmp_df.xls']
/content/drive/My Drive/X-RAY/X-Ray_1001 ['RX TORACE'] files []
/content/drive/My Drive/X-RAY/X-Ray_1001/RX TORACE ['W TORACE PA - 1'] files []
/content/drive/My Drive/X-RAY/X-Ray_1001/RX TORACE/W TORACE PA - 1 [] files ['IM-0001-25351424-0001.dcm', 'IM-0001-23249568-0001.dcm']
/content/drive/My Drive/X-RAY/X-Ray_998 ['RX TORACE'] files []
/content/drive/My Drive/X-RAY/X-Ray_998/RX TORACE ['W TORACE LAT - 2'] files []
/content/drive/My Drive/X-RAY/X-Ray_998/RX TORACE/W TORACE LAT - 2 [] files ['IM-0001-10680032-0001.dcm', 'IM-0001-10673056-0001.dcm']
/content/drive/My Drive/X-RAY/X-Ray_1002 ['RX TORACE'] files []
/content/drive/My Drive/X-RAY/X-Ray_1002/RX TORACE ['TORACE LAT - 2'] files []
/content/drive/My Drive/X-RAY/X-Ray_1002/RX TORACE/TORACE LAT - 2 [] files ['IM-0001-25381440-0001.dcm', 'IM-0001-29566752-0001.dcm']
/content/drive/My Drive/X-RAY/X-Ray_1000 ['RX

In [40]:
file_list

['/content/drive/My Drive/X-RAY/X-Ray_1001/RX TORACE/W TORACE PA - 1/IM-0001-25351424-0001.dcm',
 '/content/drive/My Drive/X-RAY/X-Ray_1001/RX TORACE/W TORACE PA - 1/IM-0001-23249568-0001.dcm',
 '/content/drive/My Drive/X-RAY/X-Ray_998/RX TORACE/W TORACE LAT - 2/IM-0001-10680032-0001.dcm',
 '/content/drive/My Drive/X-RAY/X-Ray_998/RX TORACE/W TORACE LAT - 2/IM-0001-10673056-0001.dcm',
 '/content/drive/My Drive/X-RAY/X-Ray_1002/RX TORACE/TORACE LAT - 2/IM-0001-25381440-0001.dcm',
 '/content/drive/My Drive/X-RAY/X-Ray_1002/RX TORACE/TORACE LAT - 2/IM-0001-29566752-0001.dcm',
 '/content/drive/My Drive/X-RAY/X-Ray_1000/RX TORACE/W034 Torace lat SX - 2/IM-0001-21219648-0001.dcm',
 '/content/drive/My Drive/X-RAY/X-Ray_1000/RX TORACE/W034 Torace lat SX - 2/IM-0001-21103488-0001.dcm',
 '/content/drive/My Drive/X-RAY/X-Ray_999/RX TORACE/W TORACE PA - 1/IM-0001-6439136-0001.dcm',
 '/content/drive/My Drive/X-RAY/X-Ray_999/RX TORACE/W TORACE PA - 1/IM-0001-6439456-0001.dcm']

In [41]:
def filter_images(file_name):
  ds=dcmread(file_name)
  return any(el in ds.get("SeriesDescription").lower() for el in ["pa", "p.a."])


"""
Carica un file e ritorna l'immagine e i label.
Entrambi devono essere tensori di tf.
"""
def get_image_and_labels(file_name, label):

  image_bytes = tf.io.read_file(file_name)
  print(image_bytes)
  image = tfio.image.decode_dicom_image(image_bytes, dtype=tf.uint16)
  print(image)
  image = tf.image.resize(image, (224, 224))
  print(image)
  image = tf.image.grayscale_to_rgb(image)
  print(image)
  image=tf.reshape(image, [224,224,3])
  print(image)
  image = tf.image.convert_image_dtype(image, tf.float32)
  print(image)
  label=tf.squeeze(label)
  return image, label

"""
Normalizzare o scalare l'immagine se serve!
"""
def normalize_image(img,labels):
    #img=tf.image.per_image_standardization(img)

    imagenet_mean = np.array([0.485, 0.456, 0.406])
    imagenet_std = np.array([0.229, 0.224, 0.225])
    img = (img - imagenet_mean) / imagenet_std
    return img,labels



In [42]:
def get_labels(file_name):
  folder_name = file_name.split("/")[-4] # todo farlo meglio questo
  return df[df["ID protocol"] == folder_name].iloc[:, -12:].values


filtered_file_list = [file_name for file_name in file_list if filter_images(file_name)]
corresponding_labels = [get_labels(file_name) for file_name in filtered_file_list]

In [43]:
#dataset_tf=tf.data.from_tensor_slices([file_name for file_name in file_list if filter_images(file_name)])
batch_size = 2
dataset_tf=tf.data.Dataset.from_tensor_slices((filtered_file_list, corresponding_labels))
print(dataset_tf)
dataset_tf = dataset_tf.map(get_image_and_labels,num_parallel_calls = tf.data.experimental.AUTOTUNE)
print(dataset_tf)
dataset_tf = dataset_tf.map(normalize_image,num_parallel_calls = tf.data.experimental.AUTOTUNE)
print(dataset_tf)
dataset_tf = dataset_tf.batch(batch_size, drop_remainder=True).prefetch(1)
print(dataset_tf)


<TensorSliceDataset shapes: ((), (1, 12)), types: (tf.string, tf.int64)>
Tensor("ReadFile:0", shape=(), dtype=string)
Tensor("IO>DecodeDICOMImage:0", shape=(None, None, None, None), dtype=uint16)
Tensor("resize/ResizeBilinear:0", shape=(None, 224, 224, None), dtype=float32)
Tensor("grayscale_to_rgb:0", shape=(None, 224, 224, 3), dtype=float32)
Tensor("Reshape:0", shape=(224, 224, 3), dtype=float32)
Tensor("Identity:0", shape=(224, 224, 3), dtype=float32)
<ParallelMapDataset shapes: ((224, 224, 3), (12,)), types: (tf.float32, tf.int64)>
<ParallelMapDataset shapes: ((224, 224, 3), (12,)), types: (tf.float32, tf.int64)>
<PrefetchDataset shapes: ((2, 224, 224, 3), (2, 12)), types: (tf.float32, tf.int64)>


In [44]:
dataset_tf

<PrefetchDataset shapes: ((2, 224, 224, 3), (2, 12)), types: (tf.float32, tf.int64)>

# Models

In [45]:
from tensorflow.keras.applications.densenet import DenseNet201,DenseNet121,DenseNet169
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.nasnet import NASNetMobile
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Activation, Lambda
from tensorflow.keras.models import Model

base_model = DenseNet121(include_top=False,
                             weights='imagenet',
                             input_shape=(224, 224, 3))



x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(12, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)


29089792/29084464 [==============================] - 0s 0us/step


In [46]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.optimizers import Adam

def step_decay(epoch):
  initial_lr = 1e-4
  drop = 0.1
  return initial_lr * np.power(drop,epoch)

outputFolder = '/content/drive/My Drive/'
if not os.path.exists(outputFolder):
    os.makedirs(outputFolder)
filepath=outputFolder+"/model-{epoch:02d}.hdf5"

lr_scheduler = LearningRateScheduler(step_decay)
early_stopping = EarlyStopping(monitor='val_AUC',mode='max',patience=4)
checkpoint_cb = ModelCheckpoint(filepath,save_best_only = False,save_weights_only = False,
                               save_freq='epoch',verbose=False)


opt = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999)
callbacks = [lr_scheduler,checkpoint_cb,early_stopping]

model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy','AUC'])

In [47]:
%%time
history = model.fit(dataset_tf,
          epochs=5,
          validation_data=dataset_tf,
          callbacks=callbacks,
          verbose=1,initial_epoch = 0)

Epoch 1/5
2/2 [==============================] - 22s 6s/step - loss: 0.8128 - accuracy: 0.0000e+00 - auc: 0.4259 - val_loss: 0.7890 - val_accuracy: 0.0000e+00 - val_auc: 0.4861


KeyboardInterrupt: ignored